In [64]:
import os
import numpy as np
import matplotlib.pyplot as plt
import const
from subprocess import check_output
from types import SimpleNamespace
from helper import *


In [79]:
base_path = os.path.join('..', 'KNexplosions')


In [136]:
run = 17

# get paths
sim_path   = os.path.join(base_path, 'runhr%d' % run)
shock_path = os.path.join(base_path, 'runhr%d.79' % run)

# get metadata
setup_path = os.path.join(sim_path, 'setup.f')
Mej = float(check_output(["grep", "mej=", setup_path]).decode("utf-8")[10:]) * const.M_sol
vej = float(check_output(["grep", "v(i)=", setup_path]).decode("utf-8").split('\n')[-3].split('/')[0][17:-2]) * 1e9
nH_cgm = float(check_output(["grep", "rhot=max(", setup_path]).decode("utf-8").split('/')[0][21:])

# get file list
file_list = np.array([filename for filename in os.listdir(sim_path) if filename[2:].isdigit()])
idx_sorted = np.argsort([int(filename[2:]) for filename in file_list])
file_list = file_list[idx_sorted]
nfile = len(file_list)


In [137]:
# create dictionary to store data
data = {}

# define columns and units in simulation data files
col_name_list_sim = np.array(['zone', 'Menc', 'r', 'rho', 'vel', 'unknown1', 'unknown2', 'unknown3'])
col_unit_list_sim = np.array([1, const.M_sol, 1, 1, 1, 1, 1, 1])

# get number of zones
with open(os.path.join(sim_path, file_list[0]), 'r') as f: 
    nzone = len(f.readlines())

# create empty arrays for simulation data
for col_name in col_name_list_sim: 
    data[col_name] = np.zeros((nfile, nzone))
    
# read simulation data
for i, filename in enumerate(file_list):
    with open(os.path.join(sim_path, filename), 'r') as f: lines = f.readlines()
    for j, line in enumerate(lines):
        if j >= nzone: continue # sometimes, there are empty lines at the end of a file
        line_split = line.split()
        for k, col_name in enumerate(col_name_list_sim):
            data[col_name][i, j] = float(line_split[k]) * col_unit_list_sim[k]

# define columns and units in shock data file
col_name_list_shock = np.array(['dump', 'unknown4', 'time', 'rho_shock', 'vel_shock', 'unknown5', 'unknown6', 'temp_shock'])
col_unit_list_shock = np.array([1, 1, const.day, 1, 1, 1, 1, 1])

# get number of time steps
with open(shock_path, 'r') as f:
    nstep = len(f.readlines())

# create empty arrays for shock data
for col_name in col_name_list_shock:
    data[col_name] = np.zeros(nstep)
    
# read shock data
with open(shock_path) as f: lines = f.readlines()
for j, line in enumerate(lines):
    line_split = line.split()
    for k, col_name in enumerate(col_name_list_shock):
        data[col_name][j] = float(line_split[k]) * col_unit_list_shock[k]

np.savez(sim_path, **data, Mej=Mej, vej=vej, nH_cgm=nH_cgm)
